In [20]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from scipy.optimize import minimize
from copy import deepcopy
import chainer
import chainer.links as L
import chainer.functions as F
from chainer.optimizer_hooks import WeightDecay
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [25]:
num_in = 13
num_h = 64
num_out = 1

In [26]:
d = {}

In [38]:
boston = load_boston()
x,y = boston.data,boston.target
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [15]:
diabetes = load_diabetes()
x,y = diabetes.data,diabetes.target
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [39]:
#これは正規化が必要な時に行う、一応やったほうが安全？
mn = MinMaxScaler(feature_range=(-1,1),copy = True)
x_train = mn.fit_transform(x_train)
x_test = mn.fit_transform(x_test)

x_train[x_train < -1] = -1
x_train[x_train > 1] = 1
x_test[x_test < -1] = -1
x_test[x_test > 1] = 1

In [29]:
class Net1(chainer.Chain):

    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_in, n_hidden)
            self.l2 = L.Linear(n_hidden, n_out)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        h = self.l2(h)
        return h

In [123]:
class Net2(chainer.Chain):

    def __init__(self,n_in=1,n_hidden=3,n_out=1):
        super().__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_in,n_hidden)
            self.l2 = L.Linear(n_hidden,n_hidden)
            self.l3 = L.Linear(n_hidden,n_out)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h

In [40]:
net = Net1(n_in=num_in, n_hidden=num_h, n_out=num_out)
optimizer = chainer.optimizers.Adam()
optimizer.setup(net)

In [48]:
n_epoch = 100
for epoch in range(n_epoch):
    # データセット並べ替えた順番を取得
    order = np.random.permutation(range(len(x_train)))
    for i in range(len(order)):
        # バッチを準備
        index = order[i]
        x_train_batch = x_train[index:index+1]
        y_train_batch = y_train[index:index+1].reshape(1,1)

        # 予測値を出力
        y_pred = net(x_train_batch)
        # 目的関数を適用
        loss_train_batch = F.squared_error(y_train_batch, y_pred)*0.5

        # 勾配のリセットと勾配の計算
        net.cleargrads()
        loss_train_batch.backward()

        # パラメータの更新
        optimizer.update()

In [74]:
#ここまでがN.N.の学習フェーズ
#ここからNICの計算フェーズ

In [49]:
par = np.array([0]*(num_h*(num_in+num_out+1) + num_out),dtype=np.float32) #ここで学習済パラメータを1次元化する
for i in range(num_in*num_h):
    par[i] = net.l1.W.array[i//num_in][i%num_in]
for i in range(num_in*num_h,(num_in+1)*num_h):
    par[i] = net.l1.b.array[i%num_h]
for i in range((num_in+1)*num_h,(num_in+num_out+1)*num_h):
    par[i] = net.l2.W.array[(i-(num_in+1)*num_h)//num_h][(i-(num_in+1)*num_h)%num_h]
for i in range((num_in+num_out+1)*num_h ,num_h*(num_in+num_out+1)+num_out):
    par[i] = net.l2.b.array[(i-(num_in+num_out+1)*num_h)]

In [50]:
class Net_after_optimization(chainer.Chain):
    def __init__(self,W_l1,b_l1,W_l2,b_l2,n_in,n_hidden,n_out):
        super().__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_in, n_hidden,initialW=W_l1,initial_bias=b_l1)
            self.l2 = L.Linear(n_hidden, n_out,initialW=W_l2,initial_bias=b_l2)

    def __call__(self, x):
        h = F.relu(self.l1(x))
        h = self.l2(h)
        return h

In [64]:
def loss_func(x,y,theta):
    #x:入力値、y:出力値、theta:学習後のパラメータベクトル,x,yは1次元のndarray(shape=(1,)となっている)
    #まず学習済パラメータを用いてモデルを構成し、その後損失関数を計算する
    #もし使うNNの形を変えるならこの構成部分をのコードを変える必要がある

    W1 = np.array([[0 for _ in range(num_in)] for _ in range(num_h)],dtype = np.float32)
    b1 = np.array([0 for _ in range(num_h)],dtype = np.float32)
    W2 = np.array([[0 for _ in range(num_h)] for _ in range(num_out)],dtype = np.float32)
    b2 = np.array([0 for _ in range(num_out)],dtype = np.float32)

    for i in range(num_in*num_h):
        W1[i//num_in][i%num_in] = theta[i]
    for i in range(num_in*num_h,(num_in+1)*num_h):
        b1[i%num_h] = theta[i]
    for i in range((num_in+1)*num_h, (num_in+num_out+1)*num_h):
        W2[(i-(num_in+1)*num_h)//num_h][(i-(num_in+1)*num_h)%num_h] = theta[i]
    for i in range((num_in+num_out+1)*num_h, (num_in+num_out+1)*num_h+num_out):
        b2[i-(num_in+num_out+1)*num_h] = theta[i]

    new_net = Net_after_optimization(n_in = num_in, n_hidden=num_h, n_out = num_out, W_l1=W1,b_l1=b1,W_l2=W2,b_l2=b2)
    
    y_pred = new_net(x.reshape(1,len(x_train[0]))).array[0][0]
    return 0.5*((y-y_pred)**2)

In [53]:
def compute_gradient1(loss,x,y,theta,num):
    #loss: 使うloss関数、x,y:i番目のデータ,theta:今のパラメータベクトル,num: 何番目のパラメータについて偏微分をとるか
    #出力は勾配ベクトル
    h = 1e-4
    theta1,theta2 = theta.copy(),theta.copy()
    theta1[num] += h
    theta2[num] -= h
    return (loss(x,y,theta1)-loss(x,y,theta2))/(2*h)

In [54]:
def compute_gradient2(loss,x,y,theta,num1,num2):
    h = 1e-4
    if num1 == num2:
        theta1,theta2 = theta.copy(),theta.copy()
        theta1[num1] += h
        theta2[num1] -= h
        return (loss(x,y,theta1) + loss(x,y,theta2) -2*loss(x,y,theta))/(h**2)
    else:
        theta1,theta2,theta3,theta4 = theta.copy(),theta.copy(),theta.copy(),theta.copy()
        theta1[num1],theta1[num2] = theta[num1]+h,theta[num2]+h
        theta2[num1],theta2[num2] = theta[num1]+h,theta[num2]-h
        theta3[num1],theta3[num2] = theta[num1]-h,theta[num2]+h
        theta4[num1],theta4[num2] = theta[num1]-h,theta[num2]-h
        return (loss(x,y,theta1) + loss(x,y,theta4) - loss(x,y,theta2) - loss(x,y,theta3))/(4*(h**2))

In [55]:
def G_ast(par,loss):
    #入力:学習後のパラメータ、出力:行列G^ast
    n = len(par)
    G = np.array([[0 for _ in range(n)] for _ in range(n)])
    E_x = [0]*n    #E_x[i] = i番目の偏微分の期待値
    E_xy = [[0 for _ in range(n)] for _ in range(n)]  #E_xy[i][j] = i,j番目の偏微分の期待値
    for i in range(n):
        element = 0
        for x,y in zip(x_train,y_train):
            element += compute_gradient1(loss,x,y,par,i)
        element /= float(len(x_train))
        E_x[i] = element
        
    for i in range(n):
        for j in range(n):
            if i <= j:
                element = 0
                for x,y in zip(x_train,y_train):
                    element += compute_gradient1(loss,x,y,par,i)*compute_gradient1(loss,x,y,par,j)
                element /= float(len(x_train))
                E_xy[i][j] = element
            else:
                E_xy[i][j] = E_xy[j][i]
    
    for i in range(n):
        for j in range(n):
            G[i][j] = E_xy[i][j] - E_x[i]*E_x[j]
    return G

In [56]:
def Q_ast(par,loss):
    #入力:学習後のパラメータ、出力:行列Q^ast
    n = len(par)
    Q = np.array([[0 for _ in range(n)] for _ in range(n)])
    for i in range(n):
        for j in range(n):
            element = 0
            for x,y in zip(x_train,y_train):
                element += compute_gradient2(loss,x,y,par,i,j)
            element /= float(len(x_train))
            Q[i][j] = element
    return Q

In [57]:
def D_ast(par,loss):
    res = 0
    for x,y in zip(x_train,y_train):
        res += loss(x,y,par)
    res /= float(len(x_train))
    return res

In [58]:
def NIC(par,loss):
    f1 = D_ast(par,loss)
    G = G_ast(par,loss)
    Q = Q_ast(par,loss)
    Q_inv = np.linalg.pinv(Q)
    temp = np.dot(G,Q_inv)
    return f1 + np.trace(temp)/float(len(x_train))

In [59]:
def gene_error(par,loss):
    res = 0
    for x,y in zip(x_test,y_test):
        res += loss(x,y,par)
    res /= float(len(x_train))
    return res

In [65]:
print(D_ast(par,loss_func),gene_error(par,loss_func))

2.6012687268124246 3.229359045883024
